In [1]:
import pandas as pd
import numpy as np
import os
import itertools

In [2]:
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
os.getcwd()

'c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL'

In [3]:
scores = pd.read_csv('backend/data/games/scores.csv')
scores['date'] = pd.to_datetime(scores['date'])
scores['first_score_time'] = scores['first_score_time'].apply(lambda x: int(x.split(':')[0]) + round(float(x.split(':')[1]) / 60, 2))
scores['first_score_time'] = scores['first_score_time'] + np.where(scores['first_score_quarter'] == 1, 0, 15)
scores['first_score'] = scores['team'].apply(lambda x: x.split(' ')[-1]) == scores['first_score_team']
scores = scores[['date', 'team', 'opponent', 'week', 'season', 'first_score_team', 'first_score_quarter', 'first_score_time', 'first_score', 'home_field']]
scores.tail(5)

,date,team,opponent,week,season,first_score_team,first_score_quarter,first_score_time,first_score,home_field
6571,2022-10-02,Las Vegas Raiders,Denver Broncos,4,2022,Raiders,1,9.87,True,True
6572,2022-10-02,Kansas City Chiefs,Tampa Bay Buccaneers,4,2022,Chiefs,1,14.23,True,False
6573,2022-10-02,Tampa Bay Buccaneers,Kansas City Chiefs,4,2022,Chiefs,1,14.23,False,True
6574,2022-10-03,Los Angeles Rams,San Francisco 49ers,4,2022,Rams,1,6.80,True,False
6575,2022-10-03,San Francisco 49ers,Los Angeles Rams,4,2022,Rams,1,6.80,False,True


In [4]:
df = scores.groupby(['home_field', 'first_score']).agg({'first_score': 'count'})
df = df.join(scores.groupby(['home_field'])['week'].count(), how='inner')
df['accuracy'] = round(df['first_score'] / df['week'], 2)
df.head(10)

first_score  week  accuracy
home_field first_score                             
False      False               1821  3288      0.55
           True                1467  3288      0.45
True       False               1478  3288      0.45
           True                1810  3288      0.55

In [5]:
df = scores[(scores['season'] >= 2021) & (scores['week'] >= 4)].groupby(['team', 'first_score']).agg({'first_score_time': 'mean', 'first_score': 'count'})
df = df.join(scores[scores['season'] == 2021].groupby(['team'])['week'].count(), how='inner')
df['accuracy'] = round(df['first_score'] / df['week'], 2)
df.tail(15)

first_score_time  first_score  week  \
team                     first_score                                        
New York Jets            False                9.150000            9    17   
                         True                 8.341667            6    17   
Philadelphia Eagles      False               10.897000           10    18   
                         True                 8.835000            6    18   
Pittsburgh Steelers      False                8.660000            7    18   
                         True                14.711111            9    18   
San Francisco 49ers      False               10.152727           11    20   
                         True                 7.295714            7    20   
Seattle Seahawks         False               12.372500            8    17   
                         True                10.695714            7    17   
Tampa Bay Buccaneers     False                9.827778            9    19   
                         True                 9.230000            8    19   
Tennessee Titans         False                7.465714            7    18   
                         True                 9.177778            9    18   
Washington Football Team False                9.990000           14    17   

                                      accuracy  
team                     first_score            
New York Jets            False            0.53  
                         True             0.35  
Philadelphia Eagles      False            0.56  
                         True             0.33  
Pittsburgh Steelers      False            0.39  
                         True             0.50  
San Francisco 49ers      False            0.55  
                         True             0.35  
Seattle Seahawks         False            0.47  
                         True             0.41  
Tampa Bay Buccaneers     False            0.47  
                         True             0.42  
Tennessee Titans         False            0.39  
                         True             0.50  
Washington Football Team False            0.82

In [6]:
teams = ('Las Vegas Raiders', 'Kansas City Chiefs')
first_score = (True, False)
away_first = df.loc[(teams[0], first_score[0]), :]['accuracy'] * df.loc[(teams[1], first_score[1]), :]['accuracy']
home_first = df.loc[(teams[0], first_score[1]), :]['accuracy'] * df.loc[(teams[1], first_score[0]), :]['accuracy']

In [7]:
round(away_first / (away_first + home_first) * 100, 1)

32.7

In [8]:
round(home_first / (away_first + home_first) * 100, 1)

67.3